# Demo phương pháp LSB 

In [1]:
from bitarray import bitarray
from PIL import Image    # Thư viện đọc ghi - xử lý ảnh
import numpy as np

In [6]:
def embed(msg_file, cover_img_file, num_lsbs, stego_img_file):
    '''
    msg_file: Tên file chứa tin mật
    cover_img_file: Tên file che chắn
    stego_img_file: Tên file kết quả
    num_lsbs: Số bit dùng để nhúng (Có thể dùng 1 bit cuối hoặc 2 bit cuối, ....)
    '''
    # Đọc cover img file - Đọc ảnh phủ rồi chuyển về kiểu ndarray
    cover_pixels = np.array(Image.open(cover_img_file))
    # print(type(cover_pixels))
    # print(cover_pixels.shape)
    # print(cover_pixels.dtype)
    
    # Đọc msg file - Đọc file tin mật
    with open(msg_file, 'r') as f:
        msg = f.read()
    # print(len(msg)) = 723
    
    # Chuyển msg thành msg bits
    msg_bits = bitarray()
    msg_bits.fromstring(msg)
    # print(len(msg_bits)) = 5784 = len(msg) x 8 = 723 x 8
    
    # Kiểm xem có nhúng được không? (Kích thước của dãy bit tin mật lớn hơn kích thước của ảnh phủ thì không nhúng được)
    capacity = cover_pixels.size * num_lsbs         # Lấy kích thước của mảng chứa ảnh phủ => cover_pixels.size
    if len(msg_bits) + 1 > capacity:
        print('Cannot embed')
        return False
    
    # Thêm '100...' vào msg bits
    msg_bits.extend('1' + '0' * (capacity - len(msg_bits) - 1))   # Trừ 1 vì đã thêm số 1 trong dãy 100...
   
    # Nhúng msg bits vào cover pixels
    stego_pixels = np.empty_like(cover_pixels)
    for r in range(cover_pixels.shape[0]):
        for c in range(cover_pixels.shape[1]):
            b = (r * cover_pixels.shape[1] + c) * num_lsbs
            stego_pixels[r, c] = (cover_pixels[r, c]>>num_lsbs<<num_lsbs)\
                                + int(msg_bits[b:b+num_lsbs].to01(), 2)
    
    # Ghi stego pixels xuống file
    Image.fromarray(stego_pixels).save(stego_img_file)
    
    return True
    
embed('05-msg.txt', '05-cover.bmp', 2, '05-stego.bmp')

723
5784


True

In [7]:
def extract(stego_img_file, num_lsbs, extr_msg_file):
    '''
    stego_img_file: Ảnh chứa tin mật
    num_lsbs: Số bit dùng để nhúng (Có thể dùng 1 bit cuối hoặc 2 bit cuối, ....)
    extr_msg_file: Tin mật trong văn bản
    '''
    # Đọc stego img file
    stego_pixels = np.array(Image.open(stego_img_file))
    print(stego_pixels)
    print("-----------------")
    print(stego_pixels.reshape(-1))
    
    # Rút trích msg bits
    extr_msg_bits = bitarray()
    for pixel in stego_pixels.reshape(-1):
        extr_msg_bits.extend((np.binary_repr(pixel & (2**num_lsbs-1), num_lsbs)))
    
    # Cắt đuôi '100...' ra khỏi msg bits
    extr_msg_bits = extr_msg_bits[:extr_msg_bits.to01().rfind('1')]
    
    # Chuyển msg bits thành msg
    extr_msg = extr_msg_bits.tostring()
    
    # Ghi msg xuống file
    with open(extr_msg_file, 'w') as f:
        f.write(extr_msg)
        
extract('05-stego.bmp', 2, '05-extr_msg.txt')

[[161 160 163 ... 171 152 130]
 [161 161 160 ... 170 152 129]
 [161 163 160 ... 171 154 129]
 ...
 [ 40  40  48 ... 104 100  96]
 [ 44  44  52 ... 104 104 108]
 [ 44  44  52 ... 104 104 108]]
-----------------
[161 160 163 ... 104 104 108]
